In [39]:
import os
import numpy as np
import cv2,math


In [40]:
########### NSS Spatial ##############

def NSS(saliency_map, ground_truth):
    """
    NSS is a location-based saliency metric
    This function gives NSS score for continuous prediction map and discrete ground truth locations.
    Parameters:
        saliency_map: predicted saliency map (heatmap)
        ground_truth: discrete ground truth image (binary map)
    Returns:
        NSS score (float type)
    
    """

    #Normalize the saliency map

    saliency_map_norm = (saliency_map - np.min(saliency_map))/((np.max(saliency_map)-np.min(saliency_map))*1.0)
   

    #discretize to get pixel values between 0-1
    gt = ground_truth / 255
    

    # find fixation locations
    x, y = np.where(gt == 1.0)
  
   
    # normalize using mean and standard deviation
    s_map_norm = (saliency_map_norm - np.mean(saliency_map_norm))/np.std(saliency_map_norm)

    #  calculate NSS 
    temp = []
    for i, j in zip(x, y):
        temp.append(s_map_norm[i][j])
        
    return np.mean(temp)


In [41]:
def normalize_map(s_map):

    """
    This function normalizes given heatmap using its mean and standard deviation
    Parameters:
        s_map: heatmap or binary map
    returns:
        normalized version of heatmap or binary map
    """

    # normalize the salience map (as done in MIT code)
    norm_s_map = (s_map - np.min(s_map))/((np.max(s_map)-np.min(s_map))*1.0)
    return norm_s_map


def auc_judd(s_map, gt):

    """
    This function calculates AUC-Judd value
    Parameters:
        s_map: continuous heatmap of prediction
        gt: discrete ground truth (binary map)
    returns:
        auc judd value
    """

    # ground truth is discrete, s_map is continous and normalized
    gt = gt/255

    s_map = s_map/255

    print('MAx GT:', np.max(gt))
    print('MAx smap:', np.max(s_map))

    # thresholds are calculated from the salience map, only at places where fixations are present
    thresholds = []
    for i in range(0, gt.shape[0]):
        for k in range(0, gt.shape[1]):
            if gt[i][k]>0:
                thresholds.append(s_map[i][k])
    # num fixations is no. of salience map values at gt >0
    num_fixations = np.sum(gt)


    thresholds = sorted(set(thresholds))

    # fp_list = []
    # tp_list = []
    area = []
    area.append((0.0, 0.0))
    for thresh in thresholds:
        # in the salience map, keep only those pixels with values above threshold
        temp = np.zeros(s_map.shape)
        temp[s_map >= thresh] = 1.0
        assert np.max(gt) == 1, 'something is wrong with ground truth..not discretized properly max value > 1'
        assert np.max(s_map) == 1, 'something is wrong with salience map..not normalized properly max value > 1'
        num_overlap = np.where(np.add(temp, gt)==2)[0].shape[0]
        tp = num_overlap/(num_fixations*1.0)

    # total number of pixels > threshold - number of pixels that overlap with gt / total number of non fixated pixels
        # this becomes nan when gt is full of fixations..this won't happen
        fp = (np.sum(temp) - num_overlap)/((np.shape(gt)[0] * np.shape(gt)[1]) - num_fixations)
        area.append((round(tp, 4), round(fp, 4)))
        # tp_list.append(tp)
        # fp_list.append(fp)

    # tp_list.reverse()
    # fp_list.reverse()
    area.append((1.0,1.0))
    #tp_list.append(1.0)
    #fp_list.append(1.0)
    #print tp_list
    area.sort(key = lambda x:x[0])
    tp_list =  [x[0] for x in area]
    fp_list =  [x[1] for x in area]
    return np.trapz(np.array(tp_list),np.array(fp_list))


def auc_borji(s_map, gt, splits=100, stepsize=0.1):
    
    """
    This function calculates AUC-Borji value
    Parameters:
        s_map: continuous heatmap of prediction
        gt: discrete ground truth (binary map)
        splits: splits for true positive and false positve values, calculate auc for each split
        stepsize: optional
    returns:
        auc borji value
    """

    gt = gt/255
    num_fixations = np.sum(gt)

    print('num_fixations',num_fixations)

    num_pixels = s_map.shape[0]*s_map.shape[1]
    random_numbers = []
    for i in range(0, splits):
        temp_list = []
        for k in range(0, int(num_fixations)):
            temp_list.append(np.random.randint(num_pixels))
        random_numbers.append(temp_list)

    aucs = []
    # for each split, calculate auc
    for i in random_numbers:
        r_sal_map = []
        for k in i:
            r_sal_map.append(s_map[k % s_map.shape[0]-1, int(k/s_map.shape[0])])
        # in these values, we need to find thresholds and calculate auc
        thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

        r_sal_map = np.array(r_sal_map)

    # once threshs are got
        thresholds = sorted(set(thresholds))
        area = []
        area.append((0.0,0.0))
        for thresh in thresholds:
            # in the salience map, keep only those pixels with values above threshold
            temp = np.zeros(s_map.shape)
            temp[s_map>=thresh] = 1.0
            num_overlap = np.where(np.add(temp, gt)==2)[0].shape[0]
            tp = num_overlap/(num_fixations*1.0)

            # fp = (np.sum(temp) - num_overlap)/((np.shape(gt)[0] * np.shape(gt)[1]) - num_fixations)
            # number of values in r_sal_map, above the threshold, divided by num of random locations = num of fixations
            fp = len(np.where(r_sal_map>thresh)[0])/(num_fixations*1.0)

            area.append((round(tp,4),round(fp,4)))

        area.append((1.0,1.0))
        area.sort(key = lambda x:x[0])
        tp_list =  [x[0] for x in area]
        fp_list =  [x[1] for x in area]

        aucs.append(np.trapz(np.array(tp_list),np.array(fp_list)))

    return np.mean(aucs)


def similarity(saliency_map, ground_truth):
    
    """
    Similarity is a distribution-based metric and requires continuouse heatmaps as input.
    This function calculates similarity value between predicted saliency map and ground truth
    Parameters:
        saliency_map: continuous heatmap of prediction
        ground_truth: continuous heatmap of ground truth
    returns:
        similarity
    """
    
    # here gt is not discretized nor normalized

    ground_truth = normalize_map(ground_truth)
    saliency_map = normalize_map(saliency_map)
    ground_truth = ground_truth/(np.sum(ground_truth)*1.0)
    x,y = np.where(ground_truth>0.0)
    sim = 0.0
    for i in zip(x, y):
        sim = sim + min(ground_truth[i[0], i[1]], saliency_map[i[0], i[1]])
    return sim


def cc(saliency_map, ground_truth):
    
    """
    Correlation coefficient is a distribution-based metric and requires continuous heatmaps as input.
    This function calculates correlation coefficient between predicted saliency map and ground truth
    Parameters:
        saliency_map: continuous heatmap of prediction
        ground_truth: continuous heatmap of ground truth
    returns:
       correlation coefficient (>0 posiitve and inputs are correlated)
    """
            
    s_map_norm = (saliency_map - np.mean(saliency_map))/np.std(saliency_map)
    gt_norm = (ground_truth - np.mean(ground_truth))/np.std(ground_truth)
    a = s_map_norm
    b = gt_norm
    corr = (a*b).sum() / math.sqrt((a*a).sum() * (b*b).sum())
    #r = np.corrcoef(s_map_norm, gt_norm)
    return corr


    

In [42]:
### location of ground truth and predicted heatmaps are assigned ##########
 """
 location-based metrics: NSS, AUC-Judd, AUC-Borji
 distribution-based metrics: Similarity, correlation
 
 Inputs:
     location-based metrics: heatmap of prediction and discrete ground truth
     distribution-based metrics: heatmpas of prediction and ground truth
 """ 

loc_gtdir = '/home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/sp_GT/'   #discrete gt

dist_h_gtdir = '/home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_GT_heatmaps/' # continuous gt

pdir =  '/home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_predict_heatmaps/'

nss = []
sim = []
correlation = []
aucJ = []
aucB = []


In [43]:


for i in range(0,13260, 340):

    pfile = 'predict_spatial_heatmap_'+str(i)+'.png'
    
    loc_gtfile = 'true_spatial_task_'+str(i)+'.png'
    
    dist_gtfile = 'true_spatial_heatmap_'+str(i)+'.png'
    
  
    # pfile = 'true_spatial_task_'+str(i)+'.png'
    
#     pfile = 'mouse_spatial_heatmap_'+str(i)+'.png'

        
    ppath = os.path.join(pdir, pfile)
    loc_gtpath = os.path.join(loc_gtdir, loc_gtfile)
    dist_gtpath = os.path.join(dist_h_gtdir, dist_gtfile)
    
    print('p,t ', ppath,loc_gtpath,dist_gtpath)
    predsaliencyMap = cv2.imread(ppath, cv2.IMREAD_GRAYSCALE)
    # print('sal', np.min(predsaliencyMap), np.max(predsaliencyMap))
    loc_fixsaliencyMap = cv2.imread(loc_gtpath, cv2.IMREAD_GRAYSCALE)
    dist_fixsaliencyMap = cv2.imread(dist_gtpath, cv2.IMREAD_GRAYSCALE)
    
    score = NSS(predsaliencyMap, loc_fixsaliencyMap)
    aucj = auc_judd(predsaliencyMap, loc_fixsaliencyMap)
    aucb = auc_borji(predsaliencyMap, loc_fixsaliencyMap)
       
    simi = similarity(predsaliencyMap, dist_fixsaliencyMap)
    corr = cc(predsaliencyMap, dist_fixsaliencyMap)
    
    nss.append((score))
    sim.append((simi))
    correlation.append((corr))
    aucJ.append((aucj))
    aucB.append((aucb))



p,t  /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_predict_heatmaps/predict_spatial_heatmap_0.png /home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/sp_GT/true_spatial_task_0.png /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_GT_heatmaps/true_spatial_heatmap_0.png
MAx GT: 1.0
MAx smap: 1.0
num_fixations 339.0
p,t  /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_predict_heatmaps/predict_spatial_heatmap_340.png /home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/sp_GT/true_spatial_task_340.png /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_GT_heatmaps/true_spatial_heatmap_340.png
MAx GT: 1.0
MAx smap: 1.0
num_fixations 338.0
p,t  /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_predict_heatmaps/predict_spatial_heatmap_680.png /home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/sp_GT/true_spatial_task_680.png /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_GT_heatmaps/true_spatial_heatmap_680.png
MAx GT: 1.0
MAx smap: 1.0
num_fixations 326.0
p,t  /h

In [44]:
print('NSS:', np.mean(nss),'max nSS:',np.max(nss))
print('Similarity:', np.mean(np.array(sim)),'max simi',np.max(np.array(simi)))
print('Correlation:',  np.mean(np.array(correlation)),'max corr',np.max(correlation))
print('AUC JUDD:',  np.mean(np.array(aucJ)),'max AUCJ',np.max(aucJ))
print('AUC BORJI:',  np.mean(np.array(aucB)), 'max aucB',np.max(aucB))

NSS: 5.12777711162892 max nSS: 9.036625244468146
Similarity: 0.9067478994555157 max simi 0.9310031059669749
Correlation: 0.8172741099674771 max corr 0.9321783325666908
AUC JUDD: 0.9378789858974356 max AUCJ 0.97703824
AUC BORJI: 0.9160004487179487 max aucB 0.9628434999999999


In [46]:
### location of ground truth and predicted heatmaps are assigned ##########
"""
 location-based metrics: NSS, AUC-Judd, AUC-Borji
 distribution-based metrics: Similarity, correlation
 
 Inputs:
     location-based metrics: heatmap of prediction and discrete ground truth
     distribution-based metrics: heatmpas of prediction and ground truth
""" 

loc_gtdir = '/home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/sp_GT/'   #discrete gt

dist_h_gtdir = '/home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_GT_heatmaps/' # continuous gt

pdir = dist_h_gtdir
# pdir =  '/home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_predict_heatmaps/'

nss = []
sim = []
correlation = []
aucJ = []
aucB = []

In [47]:
for i in range(0,13260, 340):

    
    loc_gtfile = 'true_spatial_task_'+str(i)+'.png'
    
    dist_gtfile = 'true_spatial_heatmap_'+str(i)+'.png'
    
    pfile = dist_gtfile
    
    
          
    ppath = os.path.join(pdir, pfile)
    loc_gtpath = os.path.join(loc_gtdir, loc_gtfile)
    dist_gtpath = os.path.join(dist_h_gtdir, dist_gtfile)
    
    print('p,t ', ppath,loc_gtpath,dist_gtpath)
    predsaliencyMap = cv2.imread(ppath, cv2.IMREAD_GRAYSCALE)
    # print('sal', np.min(predsaliencyMap), np.max(predsaliencyMap))
    loc_fixsaliencyMap = cv2.imread(loc_gtpath, cv2.IMREAD_GRAYSCALE)
    dist_fixsaliencyMap = cv2.imread(dist_gtpath, cv2.IMREAD_GRAYSCALE)
    
    score = NSS(predsaliencyMap, loc_fixsaliencyMap)
    aucj = auc_judd(predsaliencyMap, loc_fixsaliencyMap)
    aucb = auc_borji(predsaliencyMap, loc_fixsaliencyMap)
       
    simi = similarity(predsaliencyMap, dist_fixsaliencyMap)
    corr = cc(predsaliencyMap, dist_fixsaliencyMap)
    
    nss.append((score))
    sim.append((simi))
    correlation.append((corr))
    aucJ.append((aucj))
    aucB.append((aucb))



p,t  /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_GT_heatmaps/true_spatial_heatmap_0.png /home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/sp_GT/true_spatial_task_0.png /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_GT_heatmaps/true_spatial_heatmap_0.png
MAx GT: 1.0
MAx smap: 1.0
num_fixations 339.0
p,t  /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_GT_heatmaps/true_spatial_heatmap_340.png /home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/sp_GT/true_spatial_task_340.png /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_GT_heatmaps/true_spatial_heatmap_340.png
MAx GT: 1.0
MAx smap: 1.0
num_fixations 338.0
p,t  /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_GT_heatmaps/true_spatial_heatmap_680.png /home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/sp_GT/true_spatial_task_680.png /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_GT_heatmaps/true_spatial_heatmap_680.png
MAx GT: 1.0
MAx smap: 1.0
num_fixations 326.0
p,t  /home/kkjk/LSTM/Images_to_

In [48]:
print('GT NSS:', np.mean(nss),'max GT nSS:',np.max(nss))
print('GT Similarity:', np.mean(np.array(sim)),'max GT simi',np.max(np.array(simi)))
print('GT Correlation:',  np.mean(np.array(correlation)),'max GT corr',np.max(correlation))
print('GT AUC JUDD:',  np.mean(np.array(aucJ)),'max GT AUCJ',np.max(aucJ))
print('GT AUC BORJI:',  np.mean(np.array(aucB)), 'max GT aucB',np.max(aucB))

GT NSS: 6.71405487419599 max GT nSS: 10.88586268846514
GT Similarity: 0.9999999999998987 max GT simi 0.9999999999995736
GT Correlation: 1.0 max GT corr 1.0
GT AUC JUDD: 0.9859960979487181 max GT AUCJ 0.99600998
GT AUC BORJI: 0.9426810384615385 max GT aucB 0.9769485


In [50]:
### location of ground truth and predicted heatmaps are assigned ##########
"""
 location-based metrics: NSS, AUC-Judd, AUC-Borji
 distribution-based metrics: Similarity, correlation
 
 Inputs:
     location-based metrics: heatmap of prediction and discrete ground truth
     distribution-based metrics: heatmpas of prediction and ground truth
""" 

loc_gtdir = '/home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/sp_GT/'   #discrete gt

dist_h_gtdir = '/home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_GT_heatmaps/' # continuous gt


pdir =  '/home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_Mouse_heatmaps/'

nss = []
sim = []
correlation = []
aucJ = []
aucB = []

In [51]:


for i in range(0,13260, 340):

    pfile = 'mouse_spatial_heatmap_'+str(i)+'.png'
    
    loc_gtfile = 'true_spatial_task_'+str(i)+'.png'
    
    dist_gtfile = 'true_spatial_heatmap_'+str(i)+'.png'
    
  

        
    ppath = os.path.join(pdir, pfile)
    loc_gtpath = os.path.join(loc_gtdir, loc_gtfile)
    dist_gtpath = os.path.join(dist_h_gtdir, dist_gtfile)
    
    print('p,t ', ppath,loc_gtpath,dist_gtpath)
    predsaliencyMap = cv2.imread(ppath, cv2.IMREAD_GRAYSCALE)
    # print('sal', np.min(predsaliencyMap), np.max(predsaliencyMap))
    loc_fixsaliencyMap = cv2.imread(loc_gtpath, cv2.IMREAD_GRAYSCALE)
    dist_fixsaliencyMap = cv2.imread(dist_gtpath, cv2.IMREAD_GRAYSCALE)
    
    score = NSS(predsaliencyMap, loc_fixsaliencyMap)
    aucj = auc_judd(predsaliencyMap, loc_fixsaliencyMap)
    aucb = auc_borji(predsaliencyMap, loc_fixsaliencyMap)
       
    simi = similarity(predsaliencyMap, dist_fixsaliencyMap)
    corr = cc(predsaliencyMap, dist_fixsaliencyMap)
    
    nss.append((score))
    sim.append((simi))
    correlation.append((corr))
    aucJ.append((aucj))
    aucB.append((aucb))



p,t  /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_Mouse_heatmaps/mouse_spatial_heatmap_0.png /home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/sp_GT/true_spatial_task_0.png /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_GT_heatmaps/true_spatial_heatmap_0.png
MAx GT: 1.0
MAx smap: 1.0
num_fixations 339.0
p,t  /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_Mouse_heatmaps/mouse_spatial_heatmap_340.png /home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/sp_GT/true_spatial_task_340.png /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_GT_heatmaps/true_spatial_heatmap_340.png
MAx GT: 1.0
MAx smap: 1.0
num_fixations 338.0
p,t  /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_Mouse_heatmaps/mouse_spatial_heatmap_680.png /home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/sp_GT/true_spatial_task_680.png /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_GT_heatmaps/true_spatial_heatmap_680.png
MAx GT: 1.0
MAx smap: 1.0
num_fixations 326.0
p,t  /home/kkjk/LST

In [52]:
print('mouse NSS:', np.mean(nss),'max mouse nSS:',np.max(nss))
print('mouse Similarity:', np.mean(np.array(sim)),'max mouse simi',np.max(np.array(simi)))
print('mouse Correlation:',  np.mean(np.array(correlation)),'max mouse corr',np.max(correlation))
print('mouse AUC JUDD:',  np.mean(np.array(aucJ)),'max mouse AUCJ',np.max(aucJ))
print('mouse AUC BORJI:',  np.mean(np.array(aucB)), 'max mouse aucB',np.max(aucB))


mouse NSS: 2.7537702134123623 max mouse nSS: 5.68584269761266
mouse Similarity: 0.8434815655174995 max mouse simi 0.9894001065365753
mouse Correlation: 0.48307468934631925 max mouse corr 0.8526330284276868
mouse AUC JUDD: 0.8851265976923076 max mouse AUCJ 0.9762450600000001
mouse AUC BORJI: 0.8583302564102564 max mouse aucB 0.9518190000000001
